In [ ]:
# !pip install sbi

In [ ]:
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Simulating Waves

Let's say we are investigating sine waves and we are interested in finding the amplitude $\theta$ given an observed wave $x$, so $p(\theta | x)$. To solve our problem, we build a simulator that can create synthetic observations. We can think of this simulator implicitly defining a likelihood $p(x|\theta)$. This will be the basis for our inference further down. But first, let's have a look at our simulator.

In [ ]:
def wave_simulator(theta, plot=False):
    """ f(x | theta) = theta * sin(x) """
    xs = np.linspace(0,10,100) # x values
    observation = theta * torch.sin(torch.Tensor(xs))  + torch.randn(len(xs)) * 0.1 # y values
    if plot: # make it usable in interactive plot
        plt.plot(observation,label=f'$\\theta={theta}$',color='#FF1053')
        plt.ylim(-5, 5)
        plt.legend()
        plt.show()

    return observation


You can play around with the simulator and produce different observations by adjusting the $\theta$ parameter.

In [ ]:
simulation = interactive(wave_simulator, theta=(-2.0, 2.0), plot=True)
simulation

To perform Simulation-based Inference we need the simulator to produce many observations and create a synthetic dataset. So in the next step, choose a sample size $n$ that you think is sufficient and let the simulator run $n$ times. Of course, we do not want to adjust the amplitude manually, so we'll create a prior, from which we can sample the amplitude from. These are all the hyperparameters set, and sbi will use them to perform inference.

In [ ]:
simulation_hyperparameters = interactive(lambda sample_size, prior_lower, prior_upper: (sample_size, prior_lower, prior_upper),
                          sample_size=(0,1000),prior_lower=(-2,-0.1),prior_upper=(0.1,2))

simulation_hyperparameters

In [ ]:
# initiate samples 
num_samples, prior_lower, prior_upper = simulation_hyperparameters.result

samples = np.zeros((num_samples, len(np.linspace(0,10,100))))

# create prior
prior = utils.BoxUniform(low=prior_lower*torch.ones(1), high=prior_upper*torch.ones(1))

# run simulator and infer 
posterior = infer(wave_simulator, prior, method='SNPE', num_simulations=num_samples)

You have successfully used the sine wave simulator and sbi to learn a posterior. Let's have a look whether it works. You can again create a synthetic observation, and this time we will pass it to our posterior and see, what it will return.

In [ ]:
simulation

In [ ]:
sample = posterior.sample((10000,), x=simulation.result)

_ = analysis.pairplot(sample, limits=[[-2,2],[-2,2],[-2,2]], figsize=(6,6))

Are you satisfied with the conditional posterior? If not, maybe, try out different hyperparameters.